In [1]:
import tensorflow

In [2]:
import keras

Using TensorFlow backend.


import numpy as np
from keras import backend as K
from keras.layers.core import dense
from keras.optimizers import Adam
4

In [6]:
import numpy as np 
from keras import backend as K 
from keras.layers.core import Dense 
from keras.optimizers import Adam 
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import image
from keras.models import Model
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
total_gen = ImageDataGenerator(validation_split=0.3)


train_gen = data_generator.flow_from_directory(my_dir, target_size=(224,224), shuffle=False, seed=13,
                                                     class_mode='categorical', batch_size=10 subset="training")


SyntaxError: invalid syntax (<ipython-input-16-a9159502d83f>, line 5)

In [37]:
import os
import random
from shutil import copyfile
img_source_dir='C:\\Users\\Raghav'+' ' +'Chitkara\\Desktop\\mobile_net\\dataset-resized\\'
def img_train_test_split(img_source_dir, train_size):
    """
    Randomly splits images over a train and validation folder, while preserving the folder structure
    
    Parameters
    ----------
    img_source_dir : string
        Path to the folder with the images to be split. Can be absolute or relative path   
        
    train_size : float
        Proportion of the original images that need to be copied in the subdirectory in the train folder
    """    
    if not (isinstance(img_source_dir, str)):
        raise AttributeError('img_source_dir must be a string')
        
    if not os.path.exists(img_source_dir):
        raise OSError('img_source_dir does not exist')
        
    if not (isinstance(train_size, float)):
        raise AttributeError('train_size must be a float')
        
    # Set up empty folder structure if not exists
    if not os.path.exists('data'):
        os.makedirs('data')
    else:
        if not os.path.exists('data/train'):
            os.makedirs('data/train')
        if not os.path.exists('data/validation'):
            os.makedirs('data/validation')
            
    # Get the subdirectories in the main image folder
    subdirs = [subdir for subdir in os.listdir(img_source_dir) if os.path.isdir(os.path.join(img_source_dir, subdir))]

    for subdir in subdirs:
        subdir_fullpath = os.path.join(img_source_dir, subdir)
        if len(os.listdir(subdir_fullpath)) == 0:
            print(subdir_fullpath + ' is empty')
            break

        train_subdir = os.path.join('data/train', subdir)
        validation_subdir = os.path.join('data/validation', subdir)

        # Create subdirectories in train and validation folders
        if not os.path.exists(train_subdir):
            os.makedirs(train_subdir)

        if not os.path.exists(validation_subdir):
            os.makedirs(validation_subdir)

        train_counter = 0
        validation_counter = 0

        # Randomly assign an image to train or validation folder
        for filename in os.listdir(subdir_fullpath):
            if filename.endswith(".jpg") or filename.endswith(".png"): 
                fileparts = filename.split('.')

                if random.uniform(0, 1) <= train_size:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(train_subdir, str(train_counter) + '.' + fileparts[1]))
                    train_counter += 1
                else:
                    copyfile(os.path.join(subdir_fullpath, filename), os.path.join(validation_subdir, str(validation_counter) + '.' + fileparts[1]))
                    validation_counter += 1
                    
        print('Copied ' + str(train_counter) + ' images to data/train/' + subdir)
        print('Copied ' + str(validation_counter) + ' images to data/validation/' + subdir)

In [39]:
img_train_test_split(img_source_dir, 0.9)

Copied 364 images to data/train/cardboard
Copied 39 images to data/validation/cardboard
Copied 451 images to data/train/glass
Copied 50 images to data/validation/glass
Copied 373 images to data/train/metal
Copied 37 images to data/validation/metal
Copied 532 images to data/train/paper
Copied 62 images to data/validation/paper
Copied 427 images to data/train/plastic
Copied 55 images to data/validation/plastic
Copied 119 images to data/train/trash
Copied 18 images to data/validation/trash


In [40]:
img_source_dir1='C:\\Users\\Raghav'+' ' +'Chitkara\\Desktop\\mobile_net\\data_tt\\train'
img_train_test_split(img_source_dir1, 0.9)

Copied 324 images to data/train/cardboard
Copied 40 images to data/validation/cardboard
Copied 406 images to data/train/glass
Copied 45 images to data/validation/glass
Copied 338 images to data/train/metal
Copied 35 images to data/validation/metal
Copied 472 images to data/train/paper
Copied 60 images to data/validation/paper
Copied 379 images to data/train/plastic
Copied 48 images to data/validation/plastic
Copied 102 images to data/train/trash
Copied 17 images to data/validation/trash


In [44]:
train_data_dir="C:\\Users\\Raghav"+" " +"Chitkara\\Desktop\\mobile_net\\dataset-resized\\"
train_path="C:\\Users\\Raghav"+" " +"Chitkara\\Desktop\\mobile_net\\data\\train"
valid_path='C:\\Users\\Raghav'+' ' + 'Chitkara\\Desktop\\mobile_net\\data\\validation'
test_path='C:\\Users\\Raghav'+' ' + 'Chitkara\\Desktop\\mobile_net\\data\\test'

In [42]:
train_batches=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(train_path,target_size=(224,224),batch_size=10)

Found 2021 images belonging to 6 classes.


In [64]:
valid_batches=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path,target_size=(224,224),batch_size=10)
test_batches=ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input).flow_from_directory(test_path,target_size=(224,224),batch_size=10,shuffle=False)

Found 245 images belonging to 6 classes.
Found 261 images belonging to 6 classes.


In [47]:
mobile=keras.applications.mobilenet.MobileNet()
mobile.summary()

17227776/17225924 [==============================] - 17s 1us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_

In [53]:
x=mobile.layers[-6].output
predictions=Dense(6,activation='softmax')(x)
model=Model(inputs=mobile.input,output=predictions)

C:\Users\Raghav Chitkara\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [55]:
for layer in model.layers[:-5]:
    layer.trainable=False

In [56]:
model.compile(Adam(lr=0.0001),loss="categorical_crossentropy",metrics=['accuracy'])

In [57]:
model.fit_generator(train_batches,steps_per_epoch=4,validation_data=valid_batches,validation_steps=2,epochs=30,verbose=2)

Epoch 1/30
 - 11s - loss: 2.3793 - acc: 0.1000 - val_loss: 2.3166 - val_acc: 0.1000
Epoch 2/30
 - 8s - loss: 2.1478 - acc: 0.1250 - val_loss: 2.1084 - val_acc: 0.2000
Epoch 3/30
 - 7s - loss: 1.4889 - acc: 0.4250 - val_loss: 1.9108 - val_acc: 0.1500
Epoch 4/30
 - 8s - loss: 1.4022 - acc: 0.5000 - val_loss: 1.5891 - val_acc: 0.3500
Epoch 5/30
 - 7s - loss: 1.6547 - acc: 0.2750 - val_loss: 1.8296 - val_acc: 0.2500
Epoch 6/30
 - 7s - loss: 1.3212 - acc: 0.4250 - val_loss: 1.6831 - val_acc: 0.3500
Epoch 7/30
 - 8s - loss: 1.4927 - acc: 0.3750 - val_loss: 1.5721 - val_acc: 0.4500
Epoch 8/30
 - 7s - loss: 1.1483 - acc: 0.6500 - val_loss: 1.3165 - val_acc: 0.5500
Epoch 9/30
 - 7s - loss: 1.1943 - acc: 0.6250 - val_loss: 1.7412 - val_acc: 0.2500
Epoch 10/30
 - 7s - loss: 1.1176 - acc: 0.5000 - val_loss: 0.9747 - val_acc: 0.7000
Epoch 11/30
 - 7s - loss: 1.0829 - acc: 0.6000 - val_loss: 0.9239 - val_acc: 0.7500
Epoch 12/30
 - 7s - loss: 1.0491 - acc: 0.6500 - val_loss: 1.5085 - val_acc: 0.4000


In [58]:
test_labels=test_batches.classes

In [59]:
test_batches.class_indices

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}

In [65]:
prediction=model.predict_generator(test_batches,steps=1,verbose=0)

In [66]:
test_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [61]:
confusion_matrix(test_labels,prediction.argmax(axis=1))

ValueError: Found input variables with inconsistent numbers of samples: [261, 10]